<a href="https://colab.research.google.com/github/aaronkor/dlai/blob/master/ofolio_func.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Installs
# ---------------------------
!pip install quandl

# Imports
# ---------------------------

# Testing GitHub
import quandl
import pandas as pd
import numpy as np
import time
from scipy.optimize import minimize
import matplotlib.pyplot as plt

quandl.ApiConfig.api_key = 'yNJSYyossf9pcx-SEepW'

In [0]:
# inputs
tickers = ['CNP', 'F', 'WMT', 'C', 'A', 'V', 'AAL', 'HD', 'MSI', 'GM', 'GS',
           'GOOG', 'QCOM', 'CSCO', 'MGAWY', 'PSA', 'LIN', 'KOD', 'ACC', 'CBI',
           'ECOL', 'EE', 'GGG', 'GNE', 'GMO', 'H', 'HE', 'KOP', 'KS','LNKD',
          'LORL', 'MAR', 'MCRI']
# Optimizaed for period Start_Date / End_Date
start_date = '2017-4-1'
end_date = '2018-3-31'
annual_risk_free_intrest = 0.016
max_single_asset_weight = 0.125


In [0]:
def get_data(tickers,start_date,end_date):
    data_columns = ['date', 'ticker', 'adj_close']
    retrieved_data = quandl.get_table('WIKI/PRICES',
                                      ticker=tickers,
                                      qopts={'columns': data_columns},
                                      date={'gte': start_date, 'lte': end_date},
                                      paginate=True
                                      )
    return retrieved_data

def data_preperation(retrieved_data):
  # Retrieved Data Manipulation for analysis
  clean = retrieved_data.set_index('date')
  table = clean.pivot(columns='ticker')

  # Data Perperation
  returns_daily = table.pct_change()[table.shift(1).notnull()].dropna()
  tickers_retrived = returns_daily.columns.droplevel().values
#   print("Retrived adj_close date for: ", tickers_retrived)
  return(returns_daily)



In [0]:
# Portfolio
# ---------------------------

def get_portfolio(returns, weights):
  weights_arr = np.array([weights])  
  return np.dot(returns,weights_arr.T)

def get_portfolio_return(weights, returns):
  port = get_portfolio(returns, weights)
  return np.product(1 + port) - 1

def get_portfolio_risk(weights, returns):
  cov_daily = np.cov(returns, rowvar=False, ddof=0)
  sample_size = len(returns)
  weights_arr = np.array([weights])
  w_mat = np.dot(weights_arr.T, weights_arr)
  w_cov_mat = np.sum(w_mat*cov_daily)*(sample_size/(sample_size-1))
  return np.sqrt(w_cov_mat)*np.sqrt(sample_size)

def sharp_r(weights, returns):
  port_return = get_portfolio_return(weights, returns) - annual_risk_free_intrest
  port_risk = get_portfolio_risk(weights, returns)
  return port_return/port_risk

def opt_sharp_r(weights, returns):
  port_return = get_portfolio_return(weights, returns)
  port_risk = get_portfolio_risk(weights, returns)
  return -port_return/port_risk

In [0]:
def ofolio_weight (tickers,start_date,end_date,annual_risk_free_intrest,max_single_asset_weight):
  
  retrieved_data = get_data(tickers,start_date,end_date)
  data = data_preperation(retrieved_data)
  tickers_retrived = data.columns.droplevel().values
  sample_size = len(data)
  num_assets = len(tickers_retrived)
  
  eq_weights = np.full(num_assets, 1/num_assets)
  asset_weight_limit = max_single_asset_weight
  asset_short_limit = 0
  bounds = ((asset_short_limit, asset_weight_limit),) * num_assets
  
  
  maximum_sharpe = minimize(opt_sharp_r, eq_weights, method='SLSQP',
                        args=(data),
                        constraints=({'type': 'eq', 'fun': lambda weights: 1.0 - np.sum(weights)}),
                        bounds=bounds)
  
  return [[tickers_retrived,maximum_sharpe.x], get_portfolio_return(maximum_sharpe.x, data), get_portfolio_risk(maximum_sharpe.x, data)]

In [45]:
start = time.time()
w = ofolio_weight(tickers,start_date,end_date,annual_risk_free_intrest,max_single_asset_weight)
end = time.time()
print(end - start)
print(w)

2.085285186767578
[[array(['A', 'AAL', 'ACC', 'C', 'CBI', 'CNP', 'CSCO', 'ECOL', 'EE', 'F',
       'GGG', 'GM', 'GMO', 'GNE', 'GOOG', 'GS', 'H', 'HD', 'HE', 'KOP',
       'KS', 'LORL', 'MAR', 'MCRI', 'MSI', 'PSA', 'QCOM', 'V', 'WMT'],
      dtype=object), array([6.12589750e-02, 6.50958253e-14, 5.18088603e-14, 4.71293941e-14,
       1.62278418e-13, 1.59954776e-14, 4.55597410e-02, 3.53539443e-14,
       3.89378656e-15, 3.00010220e-14, 1.19093982e-13, 6.22803557e-14,
       3.12479129e-03, 1.31222918e-13, 6.74473808e-14, 7.68915827e-14,
       1.25000000e-01, 4.97620178e-02, 1.80271601e-02, 6.86576611e-14,
       1.24919488e-01, 3.71104383e-14, 1.25000000e-01, 1.25000000e-01,
       1.25000000e-01, 1.21359883e-14, 8.46381776e-14, 1.25000000e-01,
       7.23478266e-02])], 0.397473094053431, 0.12401290137958505]
